In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np

# OPT flips the script on UA and provides an answer the question of, our forecasts are uncertain, what now? 

### for example, if our concern with UA was quantifying uncertainty in stream low flows given uncertainty in our model inputs (including pumping), we can flip that with OPT to say, what is the maximum amount we can pump such that we don't draw down stream low flows beneath some threshold? 

# OPT terminology 
- __parameter__: an uncertain model input whose value we want to estimate and whose uncertainty we want to propagate to important model outputs.
- __decision variable__: a model input whose value can be "controlled" by human activity.  For example, groundwater extraction rates or surface-water structure operations.  Like a parameter, a decision variable also influences important model outputs.
- __constraint__: an uncertain model output whose real-world equivalent value has a range of "undesired values".  In management optimization, "constraints" are typically "inequality" constraints, meaning the constraint can take any value other than the undesired values.  Think "surface-water/groundwater exchange flux must be greater than XXX to support ecological flows".
- __objective function__: a (potentially nonlinear) function of the decision variables that is to be maximized or minimized, depending on the problem.  For example, in the case of groundwater extraction, the objective is to maximize the volume of groundwater extracted (subject to not violating the constraints).

In [ ]:
# simple linear programming example (from https://realpython.com/linear-programming-python/) 
from scipy.optimize import linprog

#one objective, two DVs, 5 constraints

![lp](./lp-py-eq-2.png)

In [ ]:
x = np.linspace(0,1000)
y = 20 - 2*x
plt.fill_between(x, y,1000, color='red', alpha=0.3)

x = np.linspace(0,1000)
y = (10 + 4*x)/5
plt.fill_between(x, y,1000, color='blue', alpha=0.3)

x = np.linspace(0,1000)
y = (-2 + x)/2
plt.fill_between(x, y, color='yellow', alpha=0.3)

plt.xlim(0,10)
plt.ylim(0,10)

In [ ]:
# unbounded example
obj = [-1, -2]
#      ─┬  ─┬
#       │   └┤ Coefficient for y
#       └────┤ Coefficient for x

lhs_ineq = [[ 2,  1],  # Red constraint left side
             [-4,  5],  # Blue constraint left side
             [ 1, -2]]  # Yellow constraint left side

rhs_ineq = [20,  # Red constraint right side
            10,  # Blue constraint right side
            2]  # Yellow constraint right side

In [ ]:
bnd = [(0, float("inf")),  # Bounds of x
        (0, float("inf"))]  # Bounds of y

In [ ]:
opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq,
               bounds=bnd, method="simplex")

In [ ]:
opt.fun #objective function value (if found)

In [ ]:
opt.success #boolean for if optimal solution has been found

In [ ]:
opt.x #optimal decision variables

In [ ]:
x = np.linspace(0,1000)
y = 20 - 2*x
plt.fill_between(x, y,1000, color='red', alpha=0.3)

x = np.linspace(0,1000)
y = (10 + 4*x)/5
plt.fill_between(x, y,1000, color='blue', alpha=0.3)

x = np.linspace(0,1000)
y = (-2 + x)/2
plt.fill_between(x, y, color='yellow', alpha=0.3)

plt.xlim(0,10)
plt.ylim(0,10)

plt.scatter(opt.x[0],opt.x[1],c='black')

## what abound a bounded case?

In [ ]:
bnd = [(0, 5),  # Bounds of decision variable x
        (0, 6)]  # Bounds of decision variable y

In [ ]:
opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq,
               bounds=bnd, method="simplex")

In [ ]:
opt.fun #objective function value (if found)

In [ ]:
opt.success #boolean for if optimal solution has been found

In [ ]:
opt.x #optimal decision variables

In [ ]:
x = np.linspace(0,1000)
y = 20 - 2*x
plt.fill_between(x, y,1000, color='red', alpha=0.3)

x = np.linspace(0,1000)
y = (10 + 4*x)/5
plt.fill_between(x, y,1000, color='blue', alpha=0.3)

x = np.linspace(0,1000)
y = (-2 + x)/2
plt.fill_between(x, y, color='yellow', alpha=0.3)

plt.xlim(0,10)
plt.ylim(0,10)

plt.axvline(bnd[0][1],ls='--',c='black')
plt.axhline(bnd[1][1],ls='--',c='black')

plt.scatter(opt.x[0],opt.x[1],c='black')

In [ ]:
bnd = [(0, 3),  # Bounds of decision variable x
        (0, 4)]  # Bounds of decision variable y

In [ ]:
opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq,
               bounds=bnd, method="simplex")

In [ ]:
x = np.linspace(0,1000)
y = 20 - 2*x
plt.fill_between(x, y,1000, color='red', alpha=0.3)

x = np.linspace(0,1000)
y = (10 + 4*x)/5
plt.fill_between(x, y,1000, color='blue', alpha=0.3)

x = np.linspace(0,1000)
y = (-2 + x)/2
plt.fill_between(x, y, color='yellow', alpha=0.3)

plt.xlim(0,10)
plt.ylim(0,10)

plt.axvline(bnd[0][1],ls='--',c='black')
plt.axhline(bnd[1][1],ls='--',c='black')

plt.scatter(opt.x[0],opt.x[1],c='black')

### hitting constraint or decision variable bounds is now a necessity of the simplex, which is different from UA where we might be concerned if our parameters are always hitting bounds.

### this means that we want to put bounds on our decision variables based on reality.

### for example, for maximizing pumping subject to constraint of not capturing too much streamflow, we still want to set bounds on our pumping wells based on actual pump capacity and yield. 

## but what about uncertainty??? for example, how can we estimate that maximum amount of pumping such that we don't draw stream low flows below a threshold, given uncertainty in our parameters and constraints, or in other words with a reliability of 95% ?